# Identify Songs by Lyrics

## Imports

In [ ]:
import io
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import cv2
import cloudinary
import cloudinary.uploader
from cloudinary.utils import cloudinary_url
import yt_dlp

## CONFIG

Add parent folder to python path (needed for retrieving YOUTUBE_URL from settings.py)

In [ ]:
sys.path.append(str(Path("..").resolve()))

Initialise path for .env file (located in the parent directory)

In [ ]:
from dotenv import load_dotenv

env_path = Path("..") / ".env"
load_dotenv(dotenv_path=env_path)

Cloudinary Config

In [ ]:
CLOUDINARY_CLOUD_NAME = os.getenv("CLOUDINARY_CLOUD_NAME")
CLOUDINARY_API_KEY = os.getenv("CLOUDINARY_API_KEY")
CLOUDINARY_API_SECRET = os.getenv("CLOUDINARY_API_SECRET")
CLOUDINARY_FOLDER = os.getenv("CLOUDINARY_FOLDER")

cloudinary.config( 
    cloud_name = CLOUDINARY_CLOUD_NAME, 
    api_key = CLOUDINARY_API_KEY, 
    api_secret = CLOUDINARY_API_SECRET,
    secure = True
)

Load Database URL and Create Database Engine

In [ ]:
from sqlalchemy import create_engine, text

DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)

## Get Links Without Thumbnails

In [ ]:
query = "SELECT * FROM song_youtube_links WHERE thumbnail_key IS NULL;"

df_links_without_thumbnails = pd.read_sql(query, engine)
df_links_without_thumbnails.head(3)

## Thumbnails

Helpers

In [ ]:
def bulk_add_thumbnails(df_links_without_thumbnails, max_limit=None):
    
    processed_count = 0
    last_youtube_url = None
    cached_stream_url = None
    cap = None # Persistent capture object

    # Sort by URL to ensure the cache is used
    df_todo = df_links_without_thumbnails.sort_values('url')

    # Calculate how many we actually intend to process
    total_in_df = len(df_todo)
    actual_total = min(total_in_df, max_limit) if max_limit is not None else total_in_df

    # Use enumerate to track which row we are on (1-based)
    try:
        for i, (index, row) in enumerate(df_todo.iterrows(), 1):
            if max_limit is not None and processed_count >= max_limit:
                print(f"\n✋ Reached limit of {max_limit} songs. Stopping.")
                break

            current_url = row['url']
            
            # New Video: Close old cap, get new stream_url, open new cap
            if current_url != last_youtube_url:
                if cap is not None:
                    cap.release()
                
                # Fetch stream_url once per video
                ydl_opts = {'format': 'bestvideo[height<=360]/best'}
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    info = ydl.extract_info(current_url, download=False)
                    cached_stream_url = info['url']
                
                cap = cv2.VideoCapture(cached_stream_url)
                last_youtube_url = current_url

            # --- PROGRESS DISPLAY ---
            # This shows e.g., "Song 3 of 10 (Total Queue: 45)"
            print(f"\n{'='*30}")
            print(f"📊 Progress: {processed_count + 1} of {actual_total} (Total Queue: {total_in_df - processed_count})")
            print(f"🎵 Processing: {row['title']}")
            print(f"{'='*30}")

            # 2. Get user choice for best frame
            best_frame = _select_best_youtube_frame(
                cap=cap, 
                start_time=row['start_seconds'], 
                end_time=row['end_seconds']
            )

            if best_frame is not None:
                # 3. Upload to Cloudinary
                link_id = row["id"]
                song_usage_id = row["song_usage_id"]
                public_id = f"usage_{song_usage_id}_link_{link_id}"
                thumbnail_url = _upload_thumbnail_to_cloudinary(best_frame, public_id)
                
                if thumbnail_url:
                    # 4. Update Postgres with the new URL
                    update_query = text("""
                        UPDATE song_youtube_links 
                        SET thumbnail_key = :url 
                        WHERE id = :id
                    """)
                    
                    with engine.begin() as conn:
                        conn.execute(update_query, {"url": thumbnail_url, "id": link_id})
                    
                    # Clear previous song's images so the screen stays tidy
                    clear_output(wait=True)
        
                    print(f"✨ Successfully updated thumbnail for: {row['title']}")
                    processed_count += 1
                else:
                    print(f"❌ Cloudinary upload failed for Link ID: {link_id}")
            else:
                print(f"⏭️ Skipping Link ID: {link_id}")

    finally:
        if cap is not None:
            cap.release()

    print(f"✅ Done! Successfully processed {processed_count} thumbnails.")


def _select_best_youtube_frame(cap, start_time, end_time, options_count=4):
    frames = _get_youtube_frames(cap, start_time, end_time, options_count)
    
    if not frames:
        print("❌ No frames were captured.")
        return None

    # 2. Prompt for user input
    try:
        while True:
            choice = input(f"Enter the option number (1-{len(frames)}) or 's': ")
            if choice.lower() == 's':
                return None
            choice_idx = int(choice) - 1
            if 0 <= choice_idx < len(frames):
                return frames[choice_idx]
            else:
                print(f"❗ Invalid choice. Please enter a number between 1 and {len(frames)}.")
    except ValueError:
        return None


def _get_youtube_frames(cap, start_time, end_time, options_count=4):
    duration = end_time - start_time
    time_stamps = [start_time + (duration * i // (options_count + 1)) for i in range(1, options_count + 1)]

    frames = []
    for t in time_stamps:
        cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    if frames:
        _display_thumbnail_options(frames)
    
    return frames


def _display_thumbnail_options(frames):
    total_frames = len(frames)
    fig, ax = plt.subplots(1, total_frames, figsize=(5 * total_frames, 5))
    axes = [ax] if total_frames == 1 else ax

    for i, frame in enumerate(frames):
        axes[i].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        axes[i].set_title(f"Option {i+1}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show() 
    
    # Add a tiny pause to ensure the frontend receives the image
    plt.pause(0.1) 


def display_cv2_frame(frame):
    """Helper function used for debugging"""
    if frame is None:
        print("Cannot display frame - it is None!")
        
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(rgb_frame)
    plt.title("Best frame")
    plt.axis('off')
    plt.show()


def _upload_thumbnail_to_cloudinary(frame, public_id, target_width=480):
    # 1. Resize locally first
    h, w = frame.shape[:2]
    aspect = h / w
    target_height = int(target_width * aspect)
    small_frame = cv2.resize(frame, (target_width, target_height), interpolation=cv2.INTER_AREA)

    # 2. Encode to memory buffer
    success, buffer = cv2.imencode('.jpg', small_frame, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
    if not success:
        return None

    # 3. Upload bytes to Cloudinary
    img_bytes = io.BytesIO(buffer)
    response = cloudinary.uploader.upload(
        img_bytes, 
        public_id=public_id,
        folder=CLOUDINARY_FOLDER,
        overwrite=True,
        resource_type="image"
    )
    
    return response.get("secure_url")


In [ ]:
bulk_add_thumbnails(df_links_without_thumbnails, max_limit=10)